# Lid Driven Cavity Background
### Using NVIDIA Syms we will solve the LDC problem using PINNs
#### Steps are like:
1. Generate a 2D geometry using Modulus SYM geometry Module
2. Setup the boundary conditions
3. Select the flow equations to be solved
4. Interpret diffent losses and tune the network after followed by a basic preprocessing

The problem involves a 0.1 m square cavity with stationary walls on three sides and a top wall moving right at 1 m/s. The Reynolds number, indicating flow type, is calculated based on flow speed and cavity height. For this setup, the Reynolds number is 10, indicating laminar flow.


The governing equations for fluid dynamics in this context are the **Navier-Stokes equations**, which describe the motion of viscous fluid substances. In two dimensions, the Navier-Stokes equations can be expressed as follows:

### 1. **Continuity Equation**
The continuity equation ensures mass conservation:
$$
\frac{\partial \rho}{\partial t} + \nabla \cdot (\rho \mathbf{u}) = 0
$$
Where:
- $\rho$ is the fluid density.
- $\mathbf{u} = (u, v)$ is the velocity vector, with $u$ and $v$ being the velocity components in the $x$ and $y$ directions, respectively.


### 2. **Momentum Equations**
The momentum equations describe the change in momentum due to various forces:
$$
\frac{\partial (\rho \mathbf{u})}{\partial t} + \nabla \cdot (\rho \mathbf{u} \otimes \mathbf{u}) = -\nabla p + \mu \nabla^2 \mathbf{u} + \mathbf{f}
$$
Where:
- $p$ is the pressure.
- $\mu$ is the dynamic viscosity of the fluid.
- $\nabla^2$ is the Laplacian operator, representing viscous diffusion.
- $\mathbf{f}$ represents body forces (e.g., gravity).

### Simplified Form
In many cases, particularly for incompressible flows (where the density is constant), the equations simplify to:
1. **Continuity** (for incompressible flow):
$ \nabla \cdot \mathbf{u} = 0 $

2. **Momentum**:
$$
\frac{\partial \mathbf{u}}{\partial t} + (\mathbf{u} \cdot \nabla) \mathbf{u} = -\frac{1}{\rho} \nabla p + \nu \nabla^2 \mathbf{u}
$$
Where:
- $ \nu = \frac{\mu}{\rho} $ is the kinematic viscosity.

### Summary
The Navier-Stokes equations encapsulate the fundamental principles of fluid motion, including conservation of mass and momentum. In the script you provided, these equations are represented symbolically and solved using the Modulus framework, with boundary and interior constraints applied to model the physical behavior of the fluid.